In [ ]:
# 美國通膨率 資料爬取
# 網址:https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-
url = f"https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-"
res = req.get(url,headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"})
soup = bs(res.text, "lxml")
elements = soup.select("table td")
listUSCPI = [i.get_text() for i in elements]
listColumnName = ["Year","Annual Average CPI(-U)","Annual Percent Change(rate of inflation)"]
dfuscpi = pd.DataFrame(np.array(listUSCPI).reshape(-1,3),columns = listColumnName)
dfuscpi["Year"]=dfuscpi["Year"].str.replace("\n","").astype("int")
dfuscpi["Annual Average CPI(-U)"]=dfuscpi["Annual Average CPI(-U)"].str.replace("\n","").astype("float")
dfuscpi["Annual Percent Change(rate of inflation)"]=dfuscpi["Annual Percent Change(rate of inflation)"].str.replace("\n","").str.replace("%","")
dfuscpi.loc[""]=[2023,311.4,6.4]
dfuscpi["Year"]=dfuscpi["Year"].astype("int")

In [ ]:
# thenumbers 網站上的預算和票防資料爬蟲
url = 'https://www.the-numbers.com/movie/budgets/all'
res = req.get(url,headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'})
soup = bs(res.text,"lxml")
soup.select('table td')
columns_list = ['ReleaseDate','Movie','ProductionBudget','DomesticGross','WorldwideGross']
dfsum = pd.DataFrame(np.array([i.get_text() for i in soup.select('table td')]).reshape(-1,6)[:,1:6],columns = columns_list)
index = 101
for i in range(62):
    url = f'https://www.the-numbers.com/movie/budgets/all/{index}'
    res = req.get(url,headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'})
    soup = bs(res.text,"lxml")
    soup.select('table td')
    dfsum = dfsum.append(pd.DataFrame(np.array([i.get_text() for i in soup.select('table td')]).reshape(-1,6)[:,1:6],columns = columns_list),ignore_index = True)
    index += 100

In [ ]:
# IMDB 電影評論爬蟲
# 要修正

from bs4 import BeautifulSoup as bs
import requests as req
import json, os, pprint, time, re
import pandas as pd
import numpy as np
folderPath = 'IMDB'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
row = ["Title", "Author", "Date", "Up Vote", "Total Vote", "Rating", "Review"]
def get_IMDB_review(imdb_id):
    global df

    base_url = "https://www.imdb.com/"
    url=f"https://www.imdb.com/title/{imdb_id}/reviews?ref_=tt_ql_3"
    key = ""
    res = req.get(url)
    res.encoding = 'utf-8'
    soup = bs(res.text, "lxml")
    # 拜託啦
    title = [item.select_one(".title").text for item in soup.select(".lister-item-content")]
    author = [item.select_one(".display-name-link").text for item in soup.select(".lister-item-content")]
    date = [item.select_one(".review-date").text for item in soup.select(".lister-item-content")]
    upvote = [item.select_one('.actions.text-muted').text.split(sep=" ")[20] for item in soup.select(".lister-item-content")]
    totalvote = [item.select_one('.actions.text-muted').text.split(sep=" ")[23] for item in soup.select(".lister-item-content")]
    rating=[(item.select_one("span.rating-other-user-rating > span").text if len(item.select("span.rating-other-user-rating > span"))==2 else "") for item in soup.select(".lister-item-content") ]
    review = [item.select_one('.text').text for item in soup.select(".lister-item-content")]

    # 第二波找LOADMORE資料
    load_more = soup.select_one(".load-more-data")
    flag = True
    # 找到load_more tag, 才需要往下抓
    # 第二波的第一次抓loadmore
    if load_more.text != '\n':
        ajaxurl = load_more['data-ajaxurl']
        base_url = base_url + ajaxurl + "?ref_=undefined&paginationKey="
        key = load_more['data-key']
    else :
        while flag:
            url = base_url + key
        #     print("url = ", url)
            res = req.get(url)
            res.encoding = 'utf-8'
            soup = bs(res.text, "lxml")
            title2 = [item.select_one(".title").text for item in soup.select(".lister-item-content")]
            title += title2
            author2 = [item.select_one(".display-name-link").text for item in soup.select(".lister-item-content")]
            author += author2
            date2 = [item.select_one(".review-date").text for item in soup.select(".lister-item-content")]
            date += date2
            upvote2 = [item.select_one('.actions.text-muted').text.split(sep=" ")[20] for item in soup.select(".lister-item-content")]
            upvote += upvote2
            totalvote2 = [item.select_one('.actions.text-muted').text.split(sep=" ")[23] for item in soup.select(".lister-item-content")]
            totalvote += totalvote2
            rating2 =[(item.select_one("span.rating-other-user-rating > span").text if len(item.select("span.rating-other-user-rating > span"))==2 else "") for item in soup.select(".lister-item-content") ]
            rating += rating2
            review2 = [item.select_one('.text').text for item in soup.select(".lister-item-content")]
            review += review2

            load_more = soup.select_one(".load-more-data")
            if load_more:
                key = load_more['data-key']
            else:
                flag = False
    sumlist = np.array((title, author, date, upvote, totalvote, rating, review))
    length = sumlist.T.shape[0] 
    IMDBID = np.full((length,1),imdb_id)
    convertnp = np.concatenate((IMDBID,sumlist.T),axis = 1)
    df = pd.DataFrame(convertnp,columns = ['IMDBId','title', 'author', 'date', 'upvote', 'totalvote', 'rating', 'review'])
#     df = pd.DataFrame(list(zip(title, author, date, upvote, totalvote, rating, review)), columns=row)
    df.to_csv(f"{folderPath}/{imdb_id}.csv",index = False)
#     print(df)
    print(f"{imdb_id} success")

In [ ]:
# tmdb資料爬蟲 需要tmdb key
from tmdbv3api import TMDb
from tmdbv3api import Movie

# id範圍
max_id=330093
min_id=300000
# movie_ids=[]
# # ng_list保留作預備用
# movie_ng_ids=[]

# movie_detail存放movie深層資料
tmdb = TMDb()
tmdb.api_key = 'a3c4a5cc98710d66e97acd6404ba8cd2'
api_key = 'a3c4a5cc98710d66e97acd6404ba8cd2'
movie = Movie()
movie_detail=[]

while True:
    # API請求獲取電影淺層資訊
    # 第二層要透過detail,這裡先不做
    url=f'https://api.themoviedb.org/3/movie/{max_id}?api_key={api_key}&region=US&language=en-US'
    response = req.get(url)
    
    # 淺層資訊獲取
    data = json.loads(response.text)
    
    
    if 'id' in data.keys():
        print(f'Correct movie: {max_id}')
        detail=movie.details(max_id)
        movie_detail.append({
            "movie_id": ("NaN" if detail.id==None else detail.id),
            "imdb_id": ("NaN" if detail.imdb_id==None else detail.imdb_id),
            "title": ("NaN" if detail.original_title==None else detail.original_title),
            "en_title": ("NaN" if ([i['data']['title'] for i in detail['translations']['translations'] if i['iso_3166_1'] == 'US']==[]) else [i['data']['title'] for i in detail['translations']['translations'] if i['iso_3166_1'] == 'US'][0]),
            "budget": ("NaN" if detail.budget==None else detail.budget),
            "revenue": ("NaN" if detail.revenue==None else detail.revenue),
            "original_language": ("NaN" if detail.original_language==None else detail.original_language),
            "release_date": ("NaN" if detail.release_date==None else detail.release_date),
            "avg_rating": ("NaN" if detail.vote_average==None else detail.vote_average),
            "vote_count": ("NaN" if detail.vote_count==None else detail.vote_count),
            "sequal": ("NaN" if detail.belongs_to_collection==None else detail.belongs_to_collection['name']),
            "casts": ("NaN" if detail['casts']['cast']==None else [i['name'] for i in detail['casts']['cast']]),
            "keywords": ("NaN" if detail['keywords']['keywords']==None else [i['name'] for i in detail['keywords']['keywords']]),
            "genres": ("NaN" if detail['genres']==None else [i['name'] for i in detail['genres']]),
            "production_companies": ("NaN" if detail['production_companies']==None else [i['name'] for i in detail['production_companies']]),
            "adult": ("NaN" if detail.adult==None else detail.adult),
            "overview": ("NaN" if detail.overview==None else detail.overview),
            "popularity": ("NaN" if detail.popularity==None else detail.popularity),
            "production_countries": ("NaN" if detail['production_countries']==None else [i['name'] for i in detail['production_countries']]),
            "runtime": ("NaN" if detail.runtime==None else detail.runtime),
            "status": ("NaN" if detail.status==None else detail.status),
            "tagline": ("NaN" if detail.tagline==None else detail.tagline)
        })
    
    else:
        print(f'Incorrect movie id: {max_id}')
    
    max_id -=1
    if max_id < min_id:
        break

# 把max_id原本值抓回來,以利輸出檔案的命名
max_id=1093900



with open(f"raw_detail_{min_id}~{max_id}.json","w",encoding="utf-8") as file:
    file.write(json.dumps(movie_detail,ensure_ascii=False,indent=4))